# Задание 1
(Учимся рабоать с байтовым представлением фотографии в памяти)      
Суть задания - простейшие манипуляция с байтовым представлением файла. Но сначала надо дать справку по структуре файла .bmp

Файл .bmp состоит из следующих составляющих [(вики)](https://ru.wikipedia.org/wiki/BMP):
1. BITMAPFILEHEADER - первые 14 байт
1. BITMAPINFO - в нашем случае 40 байт
1. Пиксельные данные - всё остальное

Вот у нас есть bmp картинка :

<img src="logo.bmp" width="200" height="200">

In [1]:
#С помощью википедии, поймём какая полезная информация зашита в заголовке
with open('logo.bmp', 'rb') as logo:
    logo_bytes = logo.readline()
    print('BITMAPFILEHEADER содержит 14 байт - ', logo_bytes[0:14].hex())
    print('Размер файла в байтах : ', int.from_bytes(logo_bytes[2:6], byteorder='little'))
    print('Информация про пиксели начинается с позиции : ', int.from_bytes(logo_bytes[10:14], byteorder='little'))
    print('BITMAPINFO содержит байтов:', int.from_bytes(logo_bytes[14:18], byteorder='little'))
    print('Ширина в пикселях', int.from_bytes(logo_bytes[18:22], byteorder='little'))
    print('Высота в пикселях', int.from_bytes(logo_bytes[22:26], byteorder='little', signed=True)) # Отрицательное число - означает порядок строк сверху вниз
    print('Битность на пиксель:', int.from_bytes(logo_bytes[28:30], byteorder='little')) 
    print('Cпособ хранения данных:',int.from_bytes(logo_bytes[30:34], byteorder='little')) #0 - двумерный массив
    print('Размер пиксельных данных в байтах:',int.from_bytes(logo_bytes[34:38], byteorder='little'))

BITMAPFILEHEADER содержит 14 байт -  424d8e3379000000000036000000
Размер файла в байтах :  7943054
Информация про пиксели начинается с позиции :  54
BITMAPINFO содержит байтов: 40
Ширина в пикселях 1629
Высота в пикселях 1625
Битность на пиксель: 24
Cпособ хранения данных: 0
Размер пиксельных данных в байтах: 7943000


Каждый пиксель хранится в формате rgb в 24 битах (3 байтах). Значит на каждый цвет в памяти выделяется один байт.       
То есть Каждый пиксель в файле это три числа от 0 до 255, отвечающие за цвета красный, зелёный, синий именно в таком порядке.
В конце каждой строки стоит один байт - b'\x00'

## Задание 1 (Варинат 1):   
Напишите функцию (метод) ```half_image(input_image, result_image)```
, которая заливает верхнюю половину BMP изображения чёрным цветом.    
У этой функции должно быть два аргумета:     
- ```input_image : string``` - относительный или абсолютный путь к BMP изображению описанного выше формата. Известно, что изображение имеет чётное колчиество пикселей по высоте.
- ```result_image : string``` - относительный или абсолютный путь, по которому будет записан результат выполнения функции.   
    
Эта функция (метод) должен использовать только функции стандартных библиотек вашего языка.

In [24]:
# Возможное решение на python
def half_image(input_image : str, result_image: str):
    with open(input_image, 'rb') as inp:
        header = inp.read(54)
        width = int.from_bytes(header[18:22], byteorder='little')
        heigth = int.from_bytes(header[22:26], byteorder='little', signed=True)

        with open(result_image, 'wb') as result:
            result.writelines([header])
            
            for i in range(heigth):
                if i >= heigth/2: 
                    result.writelines([b'\x00' * width*3])
                    inp.seek(width*3,1)
                else:
                    result.writelines([inp.read(width*3)])
                result.writelines([inp.read(1)])
                
half_image('logo.bmp', 'halfed_logo.bmp')

Результат:  

<img src="halfed_logo.bmp" width="200" height="200">

## Задание 1 (Варинат 2):   
Напишите функцию (метод) ```vertical_reverse_image(input_image, result_image)```
, которая изменит порядок строк изображения на противоположный.    
У этой функции должно быть два аргумета:     
- ```input_image : string``` - относительный или абсолютный путь к BMP изображению описанного выше формата.
- ```result_image : string``` - относительный или абсолютный путь, по которому будет записан результат выполнения функции.   
    
Эта функция (метод) должен использовать только функции стандартных библиотек вашего языка.


In [27]:
# Возможное решение на python
def half_image(input_image : str, result_image: str):
    with open(input_image, 'rb') as inp:
        header = inp.read(54)
        width = int.from_bytes(header[18:22], byteorder='little')
        heigth = int.from_bytes(header[22:26], byteorder='little', signed=True)

        with open(result_image, 'wb') as result:
            result.writelines([header])

            inp.seek((heigth - 1)*(width*3 + 1) + 54)
            for i in range(heigth - 1):
                result.writelines([inp.read(width*3 + 1)])
                inp.seek(-2 * (width*3 + 1), 1)

half_image('logo.bmp', 'reversed_logo.bmp')        

In [28]:
#Возможное решение 2
##TODO поменять значение высоты на такое же отрицательное

Результат: 

<img src="reversed_logo.bmp" width="200" height="200">

Одно изображение, 1600 на 1600 пикселей с 24 битовым кодированием весит целых 7,5 мегабайт!!! Это слишком много, если мы хотим, чтобы наше изображение быстро отправлялось из Орибиты с её интернетом... Давайте, попробуем сжать это изображение

Давайте заметим, что наше изображение имеет не слишком много различных цветов. Каждый пиксель кодирует 2^24 возможных цветов, хотя импользвуется явно меньше 100. Получается, что наш файл хранит излишнюю информацию, и если избавиться от части излишней информации, то мы сможем сократить размер файла.     

# Задание 2 (Вариант 1)
1. Напишите две функции ```compose(input_image, result_image)``` и ```decompose(input_image, result_image)```, которые будут сжимать и разжимать обратно файл изображения по заданному алгоритму.
    - ```input_image : string``` - относительный или абсолютный путь к BMP изображению описанного выше формата. Гарантируется, что во входном изображении не более 256 различных цветов из 24-битной rgb палитры.
    - ```result_image : string``` - относительный или абсолютный путь, по которому будет записан результат выполнения функции.   

    Результатом сжатия (функции ```compose()```) должен стать файл формата ".cmp", который получается из оригинального BMP файла по следующему алгоритму:
    1. Первые 54 байта файла .cmp такие же как и файла .bmp
    1. Следующие 256 * 3 байта идёт "таблица использованных цветов". Это последовательность цветов, которые используются в данном изображении. 
    1. Далее идёт пиксельная информация, но каждый пиксель теперь указывается не тремя байтами rgb палитры, а одним байтом - номером соответствующего цвета слева в "таблице цветов"

    Результатом разжатия (функции ```decompose()```) должно стать BMP изображение, разжатое из файла ".cmp"
1. Во сколько раз примерно наш метод сжимает такие изображения?

In [4]:
# Возможное решение на python
##TODO 

Давайте посмотрим на верхнюю пиксельную строку нашего изображения. Она же состоит из белого пикселя "ff ff ff" повторенного много раз (столько раз, сколько влезает в одну строку изображения).     
Но!     
Мы же можем сильно сокртатить запись этого изображения, если вместо текущей записи укажем что-то вроде      
**"пиксель ff ff ff стоит 1629 раз"**, или просто **"ff ff ff 1629"**.      
Вспомним теперь, что числа у нас тоже хранятся в байтах, поэтому под число повторов мы выделим один байт. Тогда запись будет выглядеть следующим образом: **"ff ff ff 256, ff ff ff 256, ...,  ff ff ff 93"**.      
Но даже такая запись сильно сократит размер нашего файла, ведь теперь первая строка будет занимать **28 вместо 4887 байт!**  

# Задание 2 (Вариант 2)
1. Напишите две функции ```compose(input_image, result_image)``` и ```decompose(input_image, result_image)```, которые будут сжимать и разжимать обратно файл изображения по заданному алгоритму.
    - ```input_image : string``` - относительный или абсолютный путь к BMP изображению описанного выше формата. Гарантируется, что во входном изображении не более 256 различных цветов из 24-битной rgb палитры.
    - ```result_image : string``` - относительный или абсолютный путь, по которому будет записан результат выполнения функции.   

    Результатом сжатия (функции ```compose()```) должен стать файл формата ".cmp", который получается из оригинального BMP файла по следующему алгоритму:
    1. Первые 54 байта файла .cmp такие же как и файла .bmp
    1. Далее идёт пиксельная информация: каждый пиксель теперь кодируется следующим образом: сначала 3 байта цвета пикселя в палитре rgb, а потом один байт - количество раз, которое этот цвет встречается подряд

    Результатом разжатия (функции ```decompose()```) должно стать BMP изображение, разжатое из файла ".cmp"
1. Во сколько раз такой метод сжимает изображения в лучшем случае? а в худшем?

In [ ]:
# Возможное решение на python
##TODO 

# Задание 3 (Вариант 1)
1. Какие недостатки вы нашли в реализованном алгоритме сжатия? Можно ли улучшить этот алгоритм?
1. Напишите функции ```super_compose(input_image, result_image)```  и ```super_decompose(input_image, result_image)```, которая исправит увиденные вами недостатки алгоритмов и опишите формат файлов, которые получатся в результате сжатия.
1. \* Как ещё можно сжать изображение так, чтобы сохранить всю изначальную информацию? Напишите функции```extra_compose(input_image, result_image)```  и ```extra_decompose(input_image, result_image)```, которая реализует ваши идеи и опишите формат файлов, которые получатся в результате сжатия.

# Задание 3 (Вариант 2)
1. Какие недостатки вы нашли в реализованном алгоритме сжатия? Можно ли улучшить этот алгоритм?     
Некоторые проблемы можно устранить если не пытаться при сжатии сохранять всю информацию, имеющуюся в изображении. Например, если нескольким пикселям, которые имели раньше разные цвета при сжатии присваивать одинаковый.
1. Напишите функцию ```super_compose(input_image, result_image)``` , которая улучшит степень сжатия файла алгоритмом из задания 2, засчёт потери части информации, и функцию ```super_decompose(input_image, result_image)```, которая восстановит изображение BMP по сохранившейся информации. Постарайтесь сделать так, чтобы изначальное и изображение в результате последовательного сжатия и разжатия не сильно отличались друг от друга.